# Материалы:

- [Luciano Ramalho - Decorators and descriptors decoded - PyCon 2017](https://youtu.be/81S01c9zytE)
- https://realpython.com/python-descriptors/
- https://docs.python.org/3/howto/descriptor.html

In [1]:
class A:
    print('A')
    class B:
        print('B')

A
B


## Data vs non-data descriptor

If your descriptor implements just `.__get__()`, then it’s said to be a `non-data descriptor`.

If it implements `.__set__()` or `.__delete__()`, then it’s said to be a `data descriptor`. Note that this difference is not just about the name, but it’s also a difference in behavior.

Data (= overriding) descriptor: всё через `__set__`, `__get__`.
Non-data descriptor: `__get__`, если только нет в `__dict__`.

In [2]:
# data descriptor
class A(object):
   def __get__(self, obj, type):
       print('hello from get A')
   def __set__(self, obj, val):
       print('hello from set A')

# non-data descriptor
class B(object):
   def __get__(self, obj, type):
       print('hello from get B')

class C(object):
   a = A()
   b = B()

In [3]:
c = C()
c.a, c.b

hello from get A
hello from get B


(None, None)

In [4]:
c.a = 0

hello from set A


In [5]:
c.a  # A.__get__ is still called!

hello from get A


In [6]:
c.b = 0

In [7]:
c.b  # B.__get__ isn't called anymore

0

## Example

https://docs.python.org/3/howto/descriptor.html

In [8]:
from abc import ABC, abstractmethod

class Validator(ABC):
    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

class OneOf(Validator):
    def __init__(self, *options):
        self.options = set(options)

    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value!r} to be one of {self.options!r}')

class Component:
    kind = OneOf('wood', 'metal', 'plastic')
    pass

    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity

c = Component('', 'wood', 0)
c.kind
Component.kind

AttributeError: 'NoneType' object has no attribute '_kind'

## Мой пример

In [ ]:
from glom import glom


class Param:
    def __init__(self, path):
        self.path = path

    def __set_name__(self, owner, name):
        self.coeff = owner.coeff
        self.name = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self

        val = glom(obj.data, tuple(self.path.split('.')))
        # setattr(obj, self.name, val)

        return val

    # def __set__(self, obj, value) -> None:
    #     pass

    def __repr__(self):
        return f"Param('{self.path}')"


class Config:
    a = Param('a')
    b = Param('b.c')

    coeff = 123

    def __init__(self, data):
        self.data = data
    
    def backup(self):
        for a, v in Config.__dict__.items():
            if isinstance(v, Param):
                setattr(self, a, getattr(self, a))


config = Config({
    'a': 10,
    'b': {'c': 20}
})


print(config.a, config.b)
Config.a
config.backup()
print(config.a, config.b)